In [1]:
import pandas as pd

from modules.feature_engineering import make_df
from modules.data_process import process_features

from modules.data_uploads import DataUploader

from modules.model import get_xgb_model, deploy_model
#from modules.evaluation import plot_tuning, plot_errors, get_var_imp
#from modules.predict import get_initial_predictions

In [2]:
# A lot of these should acually be command line args or env variables. This is a bodge.
from config import s3_model_loc, model_name, s3_predict_out

# Create model data

The cell below should do the feature engineering from the database tables. Right now we do that in R, so I will just read the features.

In [ ]:
df = make_df(48, 97, 'class')

In [2]:
df = pd.read_csv("model.csv")
df.head()

,PayeeCustomerAccount,y,Sales3,Sales14,Sales27,Sales35,Sales36,Sales48,tier3,tier14,...,Panacur_Safeguard,Total_Implants,Zuprevo,Mastitis,Biologicals,Pharmaceuticals,manual_sales,enrol_batch,inh_tier,cluster
0,10011502,2,118.77,2576.40,4663.22,4663.22,5127.94,5365.14,0,0,...,0.00,925.54,0.00,0.0,0.0,0.0,0,3,2,4
1,10378395,1,1104.50,1885.50,6068.60,8545.46,11330.24,11330.24,0,0,...,7818.39,3380.20,0.00,0.0,0.0,0.0,0,3,1,5
2,10010224,1,2763.30,4028.06,7195.88,7195.88,7195.88,7624.79,0,0,...,0.00,0.00,0.00,0.0,0.0,0.0,0,3,1,4
3,10332540,3,483.00,2711.50,6687.92,9106.22,8733.78,10136.33,0,0,...,0.00,0.00,4537.50,0.0,0.0,0.0,0,3,2,4
4,10171378,3,5464.60,10064.36,16470.26,15254.98,12701.50,14685.47,1,1,...,1072.92,4850.20,-952.54,0.0,0.0,0.0,0,3,3,4


# Pre-process data

In this case, it will be centre and scaling, fill missing values and label encoding. 

In [3]:
df = process_features(df)
df.head()

,PayeeCustomerAccount,y,Sales3,Sales14,Sales27,Sales35,Sales36,Sales48,tier3,tier14,...,Panacur_Safeguard,Total_Implants,Zuprevo,Mastitis,Biologicals,Pharmaceuticals,manual_sales,enrol_batch,inh_tier,cluster
0,10011502,2,-0.199822,-0.106558,-0.129160,-0.155089,-0.163395,-0.187132,0,0,...,-0.162464,-0.124851,-0.057681,-0.111368,-0.116307,-0.024535,0,3,2,4
1,10378395,1,-0.154124,-0.120254,-0.106030,-0.102515,-0.090560,-0.124686,0,0,...,0.692035,-0.060944,-0.057681,-0.111368,-0.116307,-0.024535,0,3,1,5
2,10010224,1,-0.077222,-0.077783,-0.087477,-0.120792,-0.139111,-0.163477,0,0,...,-0.162464,-0.148948,-0.057681,-0.111368,-0.116307,-0.024535,0,3,1,4
3,10332540,3,-0.182936,-0.103880,-0.095837,-0.094921,-0.121051,-0.137184,0,0,...,-0.162464,-0.148948,0.054447,-0.111368,-0.116307,-0.024535,0,3,2,4
4,10171378,3,0.048010,0.041874,0.065162,-0.011654,-0.074457,-0.089561,1,1,...,-0.045201,-0.022672,-0.081220,-0.111368,-0.116307,-0.024535,0,3,3,4


# Upload data to S3

In [4]:
uploader = DataUploader(df, stratify_on='y', id_col='PayeeCustomerAccount')
#uploader.save_local()
#uploader.save_s3()

In [5]:
uploader.train

,y,Sales3,Sales14,Sales27,Sales35,Sales36,Sales48,tier3,tier14,tier27,...,Panacur_Safeguard,Total_Implants,Zuprevo,Mastitis,Biologicals,Pharmaceuticals,manual_sales,enrol_batch,inh_tier,cluster
0,1,-0.205328,-0.157630,-0.139903,-0.163929,-0.176517,-0.201314,0,0,0,...,-0.162464,-0.145225,-0.057681,-0.111368,-0.116307,-0.024535,0,3,1,4
1,1,-0.205328,-0.157630,-0.205908,-0.218239,-0.223613,-0.204635,0,0,0,...,-0.162464,-0.148948,-0.057681,-0.111368,-0.116307,-0.024535,1,3,1,1
2,1,-0.188901,-0.142501,-0.182418,-0.189055,-0.192421,-0.203437,0,0,0,...,-0.162464,-0.148948,-0.057681,-0.111368,-0.116307,-0.024535,0,3,1,5
3,1,-0.177766,-0.120789,-0.153048,-0.172026,-0.172961,-0.192865,0,0,0,...,-0.162464,-0.148948,0.010654,-0.111368,-0.116307,-0.024535,1,3,1,5
4,1,-0.193904,-0.144154,-0.188931,-0.195222,-0.200854,-0.215616,0,0,0,...,-0.162464,-0.148948,-0.057681,-0.111368,-0.116307,-0.024535,0,3,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2724,4,1.119818,1.242710,1.089450,0.823754,0.692648,0.674649,2,3,3,...,4.069390,-0.148948,0.718066,-0.111368,-0.116307,-0.024535,0,3,4,3
2725,1,0.004038,-0.068108,-0.131581,-0.152120,-0.166277,-0.167236,0,0,0,...,-0.162464,0.028771,-0.057681,-0.111368,-0.116307,-0.024535,1,3,1,1
2726,1,-0.100537,-0.102917,-0.129035,-0.154986,-0.166884,-0.172806,0,0,0,...,-0.162464,-0.035992,-0.057681,-0.111368,-0.116307,-0.024535,0,3,1,5
2727,2,-0.205328,-0.157630,-0.205908,-0.218239,-0.223613,0.026213,0,0,0,...,-0.162464,-0.148948,-0.057681,-0.111368,-0.116307,-0.024535,1,27,0,1


In [6]:
uploader.val

,y,Sales3,Sales14,Sales27,Sales35,Sales36,Sales48,tier3,tier14,tier27,...,Panacur_Safeguard,Total_Implants,Zuprevo,Mastitis,Biologicals,Pharmaceuticals,manual_sales,enrol_batch,inh_tier,cluster
0,1,-0.205328,-0.137173,-0.179361,-0.168282,-0.165573,-0.185689,0,0,0,...,0.028800,-0.148948,-0.057681,-0.111368,-0.116307,-0.024535,0,3,1,5
1,3,-0.522090,-0.151050,-0.163293,-0.137320,-0.130722,-0.140233,0,0,0,...,-0.162464,-0.148948,-0.057681,-0.111368,-0.116307,-0.024535,0,3,3,5
2,1,-0.155118,-0.121783,-0.176146,-0.193750,-0.202377,-0.224367,0,0,0,...,-0.162464,-0.148948,-0.046936,-0.111368,-0.116307,-0.024535,0,3,1,3
3,1,-0.205328,-0.157630,-0.205908,-0.194261,-0.171538,-0.196875,0,0,0,...,-0.162464,-0.148948,-0.057681,-0.111368,-0.116307,-0.024535,0,3,1,6
4,1,-0.184935,-0.130822,-0.171142,-0.177767,-0.183691,-0.194891,0,0,0,...,-0.162464,-0.148948,-0.057681,-0.111368,0.010111,-0.024535,0,3,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,3,0.273053,0.198682,0.250824,0.178307,0.221917,0.217998,1,1,2,...,-0.162464,-0.148948,-0.057681,-0.111368,-0.116307,-0.024535,0,3,3,5
386,2,0.149500,0.058402,0.004360,0.002622,-0.016204,-0.058399,1,1,1,...,-0.162464,-0.088830,-0.057681,-0.111368,-0.116307,-0.024535,0,3,1,5
387,0,-0.205328,-0.157630,-0.169036,-0.187900,-0.197305,-0.219845,0,0,0,...,-0.162464,-0.148948,-0.057681,-0.111368,-0.116307,-0.024535,0,3,1,4
388,1,-0.139798,-0.129610,-0.175855,-0.179183,-0.185604,-0.134490,0,0,0,...,-0.162464,-0.148948,-0.036368,-0.111368,-0.116307,-0.024535,1,3,1,1


In [7]:
uploader.test

,Sales3,Sales14,Sales27,Sales35,Sales36,Sales48,tier3,tier14,tier27,tier35,...,Panacur_Safeguard,Total_Implants,Zuprevo,Mastitis,Biologicals,Pharmaceuticals,manual_sales,enrol_batch,inh_tier,cluster
0,1.564259,0.962335,2.785920,3.093314,2.698519,3.902207,2,3,4,5,...,-0.162464,1.489238,2.316718,-0.111368,30.436380,-0.024535,0,3,4,5
1,-0.161059,-0.132158,-0.162210,-0.182283,-0.184578,-0.208499,0,0,0,0,...,-0.125425,-0.112252,-0.057681,-0.111368,-0.116307,-0.024535,0,3,1,5
2,-0.105394,-0.028655,0.007987,-0.001607,-0.035760,-0.045581,0,1,1,1,...,-0.162464,-0.148948,0.188298,-0.111368,-0.116307,-0.024535,0,3,2,5
3,3.017471,1.220391,0.938215,0.775813,0.638384,0.525148,3,3,3,3,...,3.052402,-0.114832,-0.057681,-0.111368,-0.116307,-0.024535,0,3,3,2
4,-0.169824,-0.126612,-0.169054,-0.179929,-0.184018,-0.198749,0,0,0,0,...,-0.162464,-0.148948,-0.057681,-0.111368,-0.116307,-0.024535,0,3,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,-0.191402,-0.114463,-0.168183,-0.183537,-0.190346,-0.213642,0,0,0,0,...,0.013160,-0.148948,-0.057681,-0.111368,-0.116307,-0.024535,0,3,1,3
776,-0.004590,-0.071797,-0.126239,-0.152686,-0.166768,-0.188965,0,0,0,0,...,-0.162464,-0.036216,-0.057681,-0.111368,-0.116307,-0.024535,0,3,2,5
777,-0.205328,-0.157630,-0.115120,-0.143537,-0.073817,-0.011484,0,0,1,1,...,-0.162464,-0.002267,0.191910,-0.111368,-0.116307,-0.024535,0,3,2,5
778,-0.205328,-0.157630,-0.205908,-0.218239,-0.223613,-0.144052,0,0,0,0,...,-0.066314,-0.042038,-0.022656,-0.111368,-0.116307,-0.024535,1,3,0,1


# Train model

In [7]:
model = get_xgb_model(s3_model_loc, model_name)
model.fit({'train': uploader.s3_train_in(), 'validation': uploader.s3_val_in()})

# Deploy model

In [ ]:
xgb_predictor = deploy_model(model, batch_output=s3_predict_out)

# Make batch predicions

In [ ]:
xgb_predictor.transform(
    data=s3_data_loc / 'test', 
    data_type='S3Prefix',
    content_type='text/csv', 
    split_type='Line'
)